In [ ]:
load_or_install = function(list.of.packages){
    new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
    if(length(new.packages)) install.packages(new.packages)
    for(pkg in list.of.packages)
        library(pkg,character.only=TRUE)
}

In [ ]:
load_or_install('palmerpenguins')
penguins = penguins[complete.cases(penguins),]
d = penguins[,c('flipper_length_mm','bill_depth_mm','species')]
head(d)

In [ ]:
d = d[d$species %in% c('Adelie','Chinstrap'),]

In [ ]:
table(d$species)

In [ ]:
d$species = droplevels(d$species)

In [ ]:
table(d$species)

In [ ]:
d$species = relevel(d$species,ref="Adelie")

In [ ]:
plot(d[,1:2],col=as.numeric(d$species))

In [ ]:
table(d$species)

In [ ]:
head(d$species)

In [ ]:
table(d$species)

In [ ]:
?glm

In [ ]:
log_mod = glm(formula=species~.,data=d,family="binomial")
summary(log_mod)

recall:

$p = logistic(X\beta) \leftrightarrow logit(p) = log(p/(1-p)) = X\beta$

In [ ]:
head(predict(log_mod))## predicts the log-odds = logit(p)

In [ ]:
X_design = model.matrix(log_mod)
head(X_design)

In [ ]:
beta_hat = log_mod$coefficients
beta_hat = array(beta_hat,c(3,1))
beta_hat

In [ ]:
head(X_design%*%beta_hat)

In [ ]:
p_hats = predict(log_mod,type="response")
head(p_hats)

In [ ]:
head(predict(log_mod))
head(log(p_hats/(1-p_hats)))

In [ ]:
train_pred = as.numeric(p_hats > .5)
head(train_pred)

In [ ]:
train_pred = factor(train_pred,labels=c('Adelie','Chinstrap'))
head(train_pred)

In [ ]:
library('caret')
confusionMatrix(data=train_pred,reference=d$species)

In [ ]:
plot_fit = function(v1,v2,df=penguins,N=floor(sqrt(10000)),scaleit=FALSE,fmla='species~.'){
    train_df = df[,c('species',v1,v2)]
    if(scaleit)
        train_df[,c(v1,v2)] = scale(train_df[,c(v1,v2)])
    
    #mod = knn3(species~.,data=train_df,k=k)
    #mod = lda(formula=as.formula(fmla),data=train_df)
    #mod = qda(formula=as.formula(fmla),data=train_df)
    mod = glm(formula=as.formula(fmla),data=df,family="binomial")
    
    r1 = range(train_df[[v1]])
    r2 = range(train_df[[v2]])
    
    s1 = seq(r1[1],r1[2],length.out=N)
    s2 = seq(r2[1],r2[2],length.out=N)
    
    p_df = expand.grid(v1=s1,v2=s2)
    colnames(p_df) = c(v1,v2)
    preds = predict(mod,newdata=p_df,type='response')
    preds = factor(preds>1/2,labels=c('Adelie','Chinstrap'))
    p_df$species = preds
    
    ggplot(data=p_df,mapping=aes_string(x=v1,y=v2,fill='species',shape='species'))+geom_tile()+
        geom_point(data=train_df,size=5)
}

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=d)

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=d,
         fmla="species~flipper_length_mm+I(flipper_length_mm^2)+bill_depth_mm+I(bill_depth_mm^2)")

In [ ]:
library('palmerpenguins')
penguins = penguins[complete.cases(penguins),]
d = penguins[,c('flipper_length_mm','bill_depth_mm','species')]
d = d[d$species %in% c('Adelie','Gentoo'),]
d$species = droplevels(d$species)
head(d)

In [ ]:
log_mod = glm(formula=species~.,data=d,family="binomial")
summary(log_mod)

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=d)

# mv logistic regression

In [ ]:
library('nnet')

In [ ]:
?multinom

In [ ]:
library('palmerpenguins')
penguins = penguins[complete.cases(penguins),]
d = penguins[,c('flipper_length_mm','bill_depth_mm','species')]
d$species = relevel(d$species,ref="Adelie")

In [ ]:
head(d[sample(nrow(d)),])

In [ ]:
mod = multinom(species~.,data=d)
summary(mod)

In [ ]:
prob_preds = predict(mod,type="probs")

In [ ]:
head(prob_preds)

In [ ]:
preds = apply(prob_preds,1,which.max)
head(preds)

In [ ]:
preds = factor(preds,labels=colnames(prob_preds))
head(preds)

In [ ]:
class_preds = predict(mod,type="class")
head(class_preds)

In [ ]:
all(class_preds == preds)

In [ ]:
plot_fit = function(v1,v2,df=penguins,N=floor(sqrt(10000)),scaleit=FALSE,fmla='species~.'){
    train_df = df[,c('species',v1,v2)]
    if(scaleit)
        train_df[,c(v1,v2)] = scale(train_df[,c(v1,v2)])
    
    # fit model
    mod = multinom(formula=as.formula(fmla),data=df)
    
    r1 = range(train_df[[v1]])
    r2 = range(train_df[[v2]])
    
    s1 = seq(r1[1],r1[2],length.out=N)
    s2 = seq(r2[1],r2[2],length.out=N)
    
    p_df = expand.grid(v1=s1,v2=s2)
    colnames(p_df) = c(v1,v2)
    preds = predict(mod,newdata=p_df,type='class')
    
    p_df$species = preds
    
    ggplot(data=p_df,mapping=aes_string(x=v1,y=v2,fill='species',shape='species'))+geom_tile()+
        geom_point(data=train_df,size=5)
}

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=d)

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=d,
        fmla='species~flipper_length_mm+bill_depth_mm+I(bill_depth_mm^2)+I(flipper_length_mm^2)')